The existence and mechanisms of Chess addiction have been an everlasting question, with forum posts mentioning it on Lichess as early [as in 2014](https://lichess.org/forum/general-chess-discussion/online-chess-addiction), but even predates the online chess' era as this [Chicago reader's 1988 demonstrates](https://chicagoreader.com/news-politics/addicted-to-chess/).


Chess addiction has always been a recurring subject online, and while its formal existence should be left to rigourous studies, it has lead me to wonder more generally about how players spent their time playing on Lichess, which we will delve into.

The data was aggregated from the January 2023 Lichess rated games (freely available on the [Lichess database](https://database.lichess.org)). Games shorter than 4 plies were filtered out. For each account (which from now on I will consider equivalent to one user), I kept their their username, as well as for each time control:
*   Number of games
*   Average rating
*   Approximate time spent, which is based on the exact time control only, and [used by Lichess](https://lichess.org/faq#time-controls) to differenciate between blitz/bullet/etc: (**clock initial time in seconds) + 40 × (clock increment)**.
*   Real time spent, based on clocks. Does not take into account the use of the `+15s` button, which was considered negligible.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define global style of the plt for all graphs
plt.style.use("dark_background")
plt.rcParams['axes.facecolor'] = "#444445"
plt.rcParams["figure.facecolor"] = "#333"
plt.rcParams["grid.color"] = "#fff"
plt.rcParams["grid.linestyle"] = "-."
plt.rcParams["grid.linewidth"] = 0.2

perfs = ["ultrabullet", "bullet", "blitz", "rapid", "classical"]
colors = [ "#1f77b4","#ff7f0e","#2ca02c","#d62728","#9467bd","#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf"]
COLOR_MAP = { "ultrabullet": , "bullet": "#81b1d2", "blitz": "#fa8174", "rapid":"#fdb462", "classical":}

In [ ]:
dtypes = {"username": "string"}

for perf in perfs:
    dtypes[f"{perf}_approximate_time"] = "Int64"
    dtypes[f"{perf}_real_time"] = "Int64"
    dtypes[f"{perf}_avg_rating"] = "Int64"

def restrict_to_perf(perf: str, with_error = True):
    l = ["username", f"{perf}_games", f"{perf}_avg_rating",f"{perf}_approximate_time", f"{perf}_real_time"]
    if with_error:
        l.append(f"{perf}_relative_error")
    return l

#df = pd.read_csv("time-spent.csv",dtype=dtypes)
df = pd.read_csv("time-spent-2017-04.csv",dtype=dtypes)
df.dtypes

In [ ]:
df_time = df.copy()

def convert(c):
    df_time[c] = df_time[c].apply(lambda x: pd.to_timedelta(x,unit="S"))

for perf in perfs:
    convert(f"{perf}_approximate_time")
    convert(f"{perf}_real_time")
df_time

Here is an extract from the data

In [ ]:
print(df_time[df_time["bullet_games"] > 2].iloc[:2][restrict_to_perf("bullet",False)].to_markdown())

|    | username        |   bullet_games |   bullet_avg_rating | bullet_approximate_time   | bullet_real_time   |
|---:|:----------------|---------------:|--------------------:|:--------------------------|:-------------------|
|  0 | claujas         |              8 |                1170 | 0 days 00:18:10           | 0 days 00:29:23    |
|  8 | danial_shahmiri |             57 |                1970 | 0 days 00:48:30           | 0 days 01:05:33    |

<INSERT_REAL_NUMBER> distinct users.

In [ ]:
df_time.describe()

In [ ]:
df1 = df[["username", "bullet_games", "bullet_avg_rating", "bullet_approximate_time", "bullet_real_time"]].copy()
df1.sort_values("bullet_avg_rating", ascending=False, inplace=True)
df1.iloc[:20]

In [ ]:
df1.hist(bins=100)

In [ ]:
# plt.grid(color='#434343', linestyle='-.', linewidth=0.7)
# Only show players with more than x games
x = 100
more_x_bullet_games = df1.query(f"bullet_games>{x}").copy()
print(f"{more_x_bullet_games.shape[0]} players with more than {x} games")
more_x_bullet_games.hist(bins=100)

In [ ]:
#for (perf, color) in zip(perfs, colors):
for (perf, color) in zip(perfs, [  # [x["color"] for x in plt.rcParams["axes.prop_cycle"]] with dark_background theme
            "#8dd3c7",
            "#feffb3",
            "#bfbbd9",
            "#fa8174",
            "#81b1d2",
            "#fdb462",
            "#b3de69",
            "#bc82bd",
            "#ccebc4",
            "#ffed6f",
        ][2:]):
    print(color, perf)
    df.hist(bins=20,log=True,color=color,column=[f"{perf}_games", f"{perf}_approximate_time", f"{perf}_real_time"])

In [ ]:
# Compute the relative error of the real time spent, compared to the average computed one
df_avg_error = df.copy()
for perf in perfs:
    df_avg_error[f"{perf}_relative_error"] = (df_avg_error[f"{perf}_approximate_time"] - df_avg_error[f"{perf}_real_time"]).abs() / df_avg_error[f"{perf}_real_time"]

# Restricting to blitz to see all columns
df_avg_error[restrict_to_perf("blitz")]

In [ ]:
df_avg_error.dtypes

In [ ]:
df_avg_error[[f"{p}_relative_error" for p in perfs]].describe(percentiles=[.5, .75,.90,.95,.99])

In [ ]:
# Investing why the max error is so high

max_avg_error_first = df_avg_error.sort_values("blitz_relative_error", ascending=False)
max_avg_error_first.query("blitz_games>=2")[restrict_to_perf("blitz")][:20]

In [ ]:
# Only show players with more than x games
x = 30
for (perf, color) in zip(perfs, colors):
    more_x_games = df_avg_error.query(f"{perf}_games>{x}").copy()
    print(f"{more_x_games.shape[0]} players with more than {x} {perf} games")
    more_x_games.hist(bins=100,column=f"{perf}_relative_error",color=color)

In [ ]:
# Scatter plot, x rating, y time-spent
for (perf, color) in zip(perfs, colors):
    df.plot.scatter(x=f"{perf}_avg_rating",y=f"{perf}_real_time",color=color,s=1)

In [ ]:
# Weighted histogram. Total time per rating
fig, axs = plt.subplots(nrows=len(perfs), figsize=(6.4, 4.8 * len(perfs)))
for (perf, color, ax) in zip(perfs, colors, axs):
    ax.hist(df[f"{perf}_avg_rating"],bins=100,color=color,weights=df[f"{perf}_real_time"])
    ax.set_title(f"{perf} rating distribution, weighted by time played")